In [ ]:
import h5py

file_path = "NR_files/resultsYAC_NR.mat"   
f = h5py.File(file_path, "r")

print("Keys:", list(f.keys()))


Keys: ['#refs#', 'sentenceData']


In [ ]:
trial = f['sentenceData']   
print("Keys in sentenceData:", list(trial.keys()))

first_key = list(trial.keys())[2]
first = trial[first_key]

print(first_key,first)

Keys in sentenceData: ['allFixations', 'content', 'mean_a1', 'mean_a1_diff', 'mean_a2', 'mean_a2_diff', 'mean_b1', 'mean_b1_diff', 'mean_b2', 'mean_b2_diff', 'mean_g1', 'mean_g1_diff', 'mean_g2', 'mean_g2_diff', 'mean_t1', 'mean_t1_diff', 'mean_t2', 'mean_t2_diff', 'omissionRate', 'rawData', 'word', 'wordbounds']
mean_a1 <HDF5 dataset "mean_a1": shape (349, 1), type "|O">


In [ ]:
import h5py
import numpy as np


words = trial['allFixations']

first_entry = words[0]        
print(type(first_entry), first_entry.shape)


ref = first_entry[0]          
print(type(ref))              

fix_data = np.array(f[ref])
print(fix_data)
print(fix_data.shape)


<class 'numpy.ndarray'> (1,)
<class 'h5py.h5r.Reference'>
['duration' 'pupilsize' 'x' 'y']
(4,)


In [1]:
import pandas as pd

df = pd.read_pickle("YAC_NR_dataset_means.pkl")
print("✅ Dataset loaded successfully!")
print("Shape:", df.shape)
print("Columns:", list(df.columns))


✅ Dataset loaded successfully!
Shape: (30, 20)
Columns: ['sentence_id', 'text', 'eeg_data', 'et_data', 'mean_a1', 'mean_a1_diff', 'mean_a2', 'mean_a2_diff', 'mean_b1', 'mean_b1_diff', 'mean_b2', 'mean_b2_diff', 'mean_g1', 'mean_g1_diff', 'mean_g2', 'mean_g2_diff', 'mean_t1', 'mean_t1_diff', 'mean_t2', 'mean_t2_diff']


In [ ]:
import h5py
import numpy as np
import pandas as pd
from tqdm import tqdm

file_path = "NR_files/resultsYDG_NR.mat"

def safe_deref(f, ref):
    """Safely dereference an HDF5 reference (handle strings, objects, or arrays)."""
    try:
        obj = f[ref]
        if isinstance(obj, h5py.Dataset):
            arr = np.array(obj)
            if arr.dtype.kind in ["S", "O"]:
                try:
                    return arr.tobytes().decode("utf-8", errors="ignore")
                except Exception:
                    return arr.astype(str)
            return arr
        return None
    except Exception:
        return None


def extract_full_features(path):
    with h5py.File(path, "r") as f:
        data = f["sentenceData"]
        n = data["content"].shape[0]

        eeg_features = [
            "mean_a1", "mean_a1_diff", "mean_a2", "mean_a2_diff",
            "mean_b1", "mean_b1_diff", "mean_b2", "mean_b2_diff",
            "mean_g1", "mean_g1_diff", "mean_g2", "mean_g2_diff",
            "mean_t1", "mean_t1_diff", "mean_t2", "mean_t2_diff"
        ]

        rows = []
        print(f"📖 Extracting {n} sentences from {path}")
        for i in tqdm(range(n)):
            row = {"sentence_index": i}

            
            try:
                ref = data["content"][i][0]
                val = safe_deref(f, ref)
                if isinstance(val, np.ndarray) and val.ndim == 2:
                    row["sentence"] = "".join(chr(int(x)) for x in val.flatten())
                else:
                    row["sentence"] = str(val)
            except Exception:
                row["sentence"] = None

            
            for feat in eeg_features:
                try:
                    ref = data[feat][i][0]
                    arr = safe_deref(f, ref)
                    row[f"{feat}_array"] = arr
                    row[f"{feat}_mean"] = np.mean(arr) if arr is not None else np.nan
                except Exception:
                    row[f"{feat}_array"] = None
                    row[f"{feat}_mean"] = np.nan

            
            try:
                ref = data["allFixations"][i][0]
                fix_ref = f[ref]
                fixation_fields = ["duration", "pupilsize", "x", "y"]
                fix_data = {}

                for field in fixation_fields:
                    if field in fix_ref:
                        fix_data[field] = np.array(fix_ref[field]).flatten()
                    else:
                        fix_data[field] = np.array([])

                
                if len(fix_data["x"]) > 0:
                    fix_df = pd.DataFrame(fix_data)
                    row["n_fixations"] = len(fix_df)
                    row["mean_fix_duration"] = np.mean(fix_df["duration"])
                    row["mean_pupilsize"] = np.mean(fix_df["pupilsize"])
                    row["fixations"] = fix_df.to_dict(orient="list")
                else:
                    row["n_fixations"] = np.nan
                    row["mean_fix_duration"] = np.nan
                    row["mean_pupilsize"] = np.nan
                    row["fixations"] = None

            except Exception:
                row["fixations"] = None
                row["n_fixations"] = np.nan
                row["mean_fix_duration"] = np.nan
                row["mean_pupilsize"] = np.nan

            
            try:
                ref = data["rawData"][i][0]
                eeg_raw = safe_deref(f, ref)
                row["eeg_raw"] = eeg_raw
            except Exception:
                row["eeg_raw"] = None


            
            try:
                ref = data["omissionRate"][i][0]
                om = safe_deref(f, ref)
                row["omission_rate"] = float(om) if om is not None else np.nan
            except Exception:
                row["omission_rate"] = np.nan

            rows.append(row)
 
    df = pd.DataFrame(rows)
    return df



df = extract_full_features(file_path)


df.to_pickle("YDG_NR_full.pkl")  
df.to_csv("YDG_NR_summary.csv", index=False)

print("✅ Extraction complete!")
print(df.head(5))


📖 Extracting 349 sentences from NR_files/resultsYDG_NR.mat


  0%|          | 0/349 [00:00<?, ?it/s]/tmp/ipykernel_7425/3766449585.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  row["omission_rate"] = float(om) if om is not None else np.nan
/tmp/ipykernel_7425/3766449585.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  row["omission_rate"] = float(om) if om is not None else np.nan
/tmp/ipykernel_7425/3766449585.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  row["omission_rate"] = float(om) if om is not Non

✅ Extraction complete!
   sentence_index                                           sentence  \
0               0  Henry Ford, with his son Edsel, founded the Fo...   
1               1  After this initial success, Ford left Edison I...   
2               2  With his interest in race cars, he formed a se...   
3               3  During this period, he personally drove his Qu...   
4               4  Ford was forced out of the company by the inve...   

                                       mean_a1_array  mean_a1_mean  \
0  [[1.0502833124750248], [1.389041244581857], [1...      1.848577   
1  [[1.1150550520367795], [1.4230471926369088], [...      1.782254   
2  [[0.7528393710930602], [1.1650646771218478], [...      1.668801   
3  [[0.8779735631643725], [1.23905766253874], [1....      1.805160   
4  [[0.9048654056240142], [1.3037000412338653], [...      1.848331   

                                  mean_a1_diff_array  mean_a1_diff_mean  \
0  [[-0.1177022222318388], [-0.18439846846115937

In [4]:
df1=pd.read_pickle("YDG_NR_full.pkl")
df1.head()

,sentence_index,sentence,mean_a1_array,mean_a1_mean,mean_a1_diff_array,mean_a1_diff_mean,mean_a2_array,mean_a2_mean,mean_a2_diff_array,mean_a2_diff_mean,...,mean_t2_array,mean_t2_mean,mean_t2_diff_array,mean_t2_diff_mean,n_fixations,mean_fix_duration,mean_pupilsize,fixations,eeg_raw,omission_rate
0,0,"Henry Ford, with his son Edsel, founded the Fo...","[[1.0502833124750248], [1.389041244581857], [1...",1.848577,"[[-0.1177022222318388], [-0.18439846846115937]...",-0.020540,"[[1.2219119265466973], [1.4150300952892994], [...",2.377014,"[[0.0382449532962571], [-0.3516655856744486], ...",0.067148,...,"[[0.8786145028394461], [1.6716294777886311], [...",2.030498,"[[0.11530522428417456], [0.061425567830284855]...",0.063752,23,103.478261,3594.739130,"{'duration': [35.0, 30.0, 97.0, 39.0, 139.0, 9...","[[1.86715616823196, 2.768817558490064, 2.84297...",0.440000
1,1,"After this initial success, Ford left Edison I...","[[1.1150550520367795], [1.4230471926369088], [...",1.782254,"[[-0.09369517089957102], [-0.17640463947922547...",0.197496,"[[1.129801401818082], [1.3278113089471397], [1...",2.167276,"[[0.05948145132192917], [-0.13813625938530405]...",0.151998,...,"[[0.9763304978417832], [1.6316800187171587], [...",1.962996,"[[-0.03310450938520093], [-0.00582412697070811...",0.158979,12,117.000000,3515.250000,"{'duration': [84.0, 67.0, 49.0, 135.0, 193.0, ...","[[0.09762623831958411, 2.5837543854022673, 3.1...",0.470588
2,2,"With his interest in race cars, he formed a se...","[[0.7528393710930602], [1.1650646771218478], [...",1.668801,"[[0.03530195056459717], [0.13781344818283936],...",0.088995,"[[1.0672039177588806], [1.4150359462914686], [...",1.993907,"[[-0.15112787668434358], [-0.2020157914012053]...",0.161066,...,"[[0.9451184419408172], [1.3366520507988848], [...",1.821082,"[[-0.23885983321609305], [-0.1284573236202301]...",-0.099627,11,117.000000,3515.909091,"{'duration': [123.0, 89.0, 70.0, 118.0, 147.0,...","[[-3.4174784133906333, -1.4030982316121745, 0....",0.466667
3,3,"During this period, he personally drove his Qu...","[[0.8779735631643725], [1.23905766253874], [1....",1.805160,"[[0.04605465270615949], [-0.07714723305014282]...",0.010958,"[[0.9163909527744442], [1.3008503473961375], [...",2.138230,"[[0.03308269665819863], [-0.0706858602122119],...",-0.127438,...,"[[1.0495866487099508], [1.4267104521425136], [...",2.083686,"[[0.22704672377405388], [-0.1921228774048992],...",-0.182845,29,109.344828,3451.034483,"{'duration': [109.0, 43.0, 99.0, 54.0, 136.0, ...","[[1.4199509173445823, 2.1346979772927237, 1.47...",0.259259
4,4,Ford was forced out of the company by the inve...,"[[0.9048654056240142], [1.3037000412338653], [...",1.848331,"[[0.08210009685194375], [-0.08582585151327027]...",0.253776,"[[1.017010431225281], [1.3449430914014566], [1...",2.019807,"[[0.025979405598686056], [-0.08728709958338177...",0.236723,...,"[[1.012805617827837], [1.535374205729871], [1....",1.977891,"[[0.11624602322295807], [-0.08825631278711354]...",0.139991,27,107.629630,3447.185185,"{'duration': [81.0, 123.0, 103.0, 111.0, 79.0,...","[[-1.7765591460112988, 1.9648168988731622, 6.0...",0.347826


In [ ]:
import os
import pandas as pd
from tqdm import tqdm


base_paths = {
    "NR": "NR_files",
    "TSR": "TSR_files"
}

all_dfs = []

for condition, folder in base_paths.items():
    files = sorted([f for f in os.listdir(folder) if f.endswith(".mat")])
    
    print(f"\n📂 Processing {condition} files ({len(files)} found)")
    for fname in tqdm(files, desc=f"Extracting {condition}"):
        file_path = os.path.join(folder, fname)
        
        try:
            df = extract_full_features(file_path)
            
            subject_id = fname.split("_")[0].replace("results", "")
            
            df["subject_id"] = subject_id
            df["condition"] = condition
            
            all_dfs.append(df)
        except Exception as e:
            print(f"❌ Error processing {fname}: {e}")

final_df = pd.concat(all_dfs, ignore_index=True)

final_df.to_pickle("Final_Fused_Dataset.pkl")
final_df.to_csv("Final_Fused_Dataset.csv", index=False)

print("\n✅ Final combined dataset created successfully!")
print(f"Total sentences: {len(final_df)}")
print(final_df.head(10))



📂 Processing NR files (4 found)


Extracting NR:   0%|          | 0/4 [00:00<?, ?it/s]

📖 Extracting 349 sentences from NR_files/resultsYAC_NR.mat


/tmp/ipykernel_7425/3766449585.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  row["omission_rate"] = float(om) if om is not None else np.nan
/tmp/ipykernel_7425/3766449585.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  row["omission_rate"] = float(om) if om is not None else np.nan
/tmp/ipykernel_7425/3766449585.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  row["omission_rate"] = float(om) if om is not None else np.nan
/tmp/ipykernel_7425/37664

📖 Extracting 349 sentences from NR_files/resultsYAK_NR.mat


...

📖 Extracting 390 sentences from TSR_files/resultsYDG_TSR.mat


/tmp/ipykernel_7425/3766449585.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  row["omission_rate"] = float(om) if om is not None else np.nan
/tmp/ipykernel_7425/3766449585.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  row["omission_rate"] = float(om) if om is not None else np.nan
/tmp/ipykernel_7425/3766449585.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  row["omission_rate"] = float(om) if om is not None else np.nan
/tmp/ipykernel_7425/37664

📖 Extracting 390 sentences from TSR_files/resultsYFS_TSR.mat


/tmp/ipykernel_7425/3766449585.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  row["omission_rate"] = float(om) if om is not None else np.nan
/tmp/ipykernel_7425/3766449585.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  row["omission_rate"] = float(om) if om is not None else np.nan
/tmp/ipykernel_7425/3766449585.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  row["omission_rate"] = float(om) if om is not None else np.nan
/tmp/ipykernel_7425/37664


✅ Final combined dataset created successfully!
Total sentences: 2956
   sentence_index                                           sentence  \
0               0  Henry Ford, with his son Edsel, founded the Fo...   
1               1  After this initial success, Ford left Edison I...   
2               2  With his interest in race cars, he formed a se...   
3               3  During this period, he personally drove his Qu...   
4               4  Ford was forced out of the company by the inve...   
5               5  In 1891, Ford became an engineer with the Edis...   
6               6  These experiments culminated in 1896 with the ...   
7               7  Ford was born on a prosperous farm in Springwe...   
8               8  In the years between the wars, Henry Ford supp...   
9               9  There is also some evidence that Henry Ford ga...   

                                       mean_a1_array  mean_a1_mean  \
0  [[0.09175936565400834], [0.16001836390609042],...      0.424368 

In [6]:
df2=pd.read_pickle("Final_Fused_Dataset.pkl")
df2.head()

,sentence_index,sentence,mean_a1_array,mean_a1_mean,mean_a1_diff_array,mean_a1_diff_mean,mean_a2_array,mean_a2_mean,mean_a2_diff_array,mean_a2_diff_mean,...,mean_t2_diff_array,mean_t2_diff_mean,n_fixations,mean_fix_duration,mean_pupilsize,fixations,eeg_raw,omission_rate,subject_id,condition
0,0,"Henry Ford, with his son Edsel, founded the Fo...","[[0.09175936565400834], [0.16001836390609042],...",0.424368,"[[0.07698649400576568], [0.08971803879267441],...",0.026597,"[[0.11026355761637595], [0.1856817389554981], ...",0.508086,"[[0.09163493243155915], [0.09607452790123273],...",0.058826,...,"[[0.09487887583647947], [0.10019126637036306],...",0.057405,22.0,136.181818,3164.909091,"{'duration': [40.0, 66.0, 51.0, 66.0, 147.0, 1...","[[0.04213374758716981, -0.5965079873535998, -0...",0.280000,YAC,NR
1,1,"After this initial success, Ford left Edison I...","[[0.10916828444809953], [0.20571546657057854],...",0.488741,"[[0.10778317034238896], [0.08628652613836782],...",-0.004890,"[[0.11665301860467002], [0.17770767143332836],...",0.520193,"[[0.09832936730703978], [0.0918771422591637], ...",0.025099,...,"[[0.12132096503337622], [0.10018725925661351],...",0.068776,34.0,100.382353,3159.588235,"{'duration': [109.0, 99.0, 130.0, 78.0, 73.0, ...","[[-0.30240420718598404, 2.473454223854762, 0.7...",0.117647,YAC,NR
2,2,"With his interest in race cars, he formed a se...","[[0.1282572907181469], [0.18347963580394402], ...",0.476327,"[[0.11580894870802821], [0.09893541899035938],...",0.003627,"[[0.1336299749877231], [0.21507017599430173], ...",0.550162,"[[0.14794574515680628], [0.10475261185639084],...",0.038220,...,"[[0.15906786186372845], [0.12429980981657077],...",0.017143,17.0,133.647059,2905.117647,"{'duration': [59.0, 155.0, 82.0, 81.0, 116.0, ...","[[0.012137721778602284, 0.09052314069747013, 0...",0.266667,YAC,NR
3,3,"During this period, he personally drove his Qu...","[[0.10514496411433202], [0.1789325352204017], ...",0.466177,"[[0.09518923821752304], [0.11479119669414187],...",0.070933,"[[0.11277224576373593], [0.20397987312715135],...",0.526020,"[[0.10354709690123567], [0.09660408305300877],...",0.028654,...,"[[0.1108656998440487], [0.13586202012784104], ...",0.154881,35.0,120.228571,2972.285714,"{'duration': [135.0, 73.0, 166.0, 130.0, 107.0...","[[-0.49631934269141675, 1.235886643666224, 0.0...",0.222222,YAC,NR
4,4,Ford was forced out of the company by the inve...,"[[0.12155919951726414], [0.182895595436164], [...",0.493315,"[[0.10302325538581314], [0.059842672009380385]...",-0.016555,"[[0.12367994580050415], [0.19443048708496977],...",0.511133,"[[0.1028522123443848], [0.07588851580843561], ...",0.003936,...,"[[0.11301876200452254], [0.07054451138992693],...",0.083583,26.0,140.384615,2762.961538,"{'duration': [66.0, 15.0, 71.0, 61.0, 222.0, 9...","[[-0.4828231257567009, -0.45618668029047826, -...",0.217391,YAC,NR
